In [62]:
import os
import zipfile
import pandas as pd
import basedosdados as bd

INPUT = os.path.join(os.getcwd(), "input")
OUTPUT = os.path.join(os.getcwd(), "output")

os.makedirs(INPUT, exist_ok=True)
os.makedirs(OUTPUT, exist_ok=True)

In [63]:
def read_sheet(sheet_name: str, skiprows: int = 8) -> pd.DataFrame:
    return pd.read_excel(
        os.path.join(
            INPUT,
            "tx_rend_brasil_regioes_ufs_esp.xlsx"
        ),
        skiprows=skiprows,
        sheet_name=sheet_name
    )

In [64]:
excel_data = pd.ExcelFile(os.path.join(
            INPUT,
            "tx_rend_brasil_regioes_ufs_esp.xlsx"
        ))

# Get the sheet names
sheet_names = excel_data.sheet_names

In [65]:
dfs = {
    sheet_name: read_sheet(sheet_name)
    for sheet_name in sheet_names
}

In [66]:
dfs

{'BRASIL_REGIOES_UFS ':                                    NU_ANO_CENSO           UNIDGEO  \
 0                                          2007            Brasil   
 1                                          2007             Norte   
 2                                          2007          Nordeste   
 3                                          2007           Sudeste   
 4                                          2007               Sul   
 ..                                          ...               ...   
 492                                        2021       Mato Grosso   
 493                                        2021             Goiás   
 494                                        2021  Distrito Federal   
 495                                         NaN               NaN   
 496  Fonte: Censo da Educação Básica 2021/INEP.               NaN   
 
     NO_CATEGORIA NO_DEPENDENCIA  1_CAT_FUN  1_CAT_FUN_AI  1_CAT_FUN_AF  \
 0          Total          Total       73.1          71.6   

In [67]:
for sheet_name, df in dfs.items():
    print(f"Sheet: {sheet_name}")
    print(df.columns)  # This will print the column names of each DataFrame
    print()  # Adds a blank line for readability


Sheet: BRASIL_REGIOES_UFS 
Index(['NU_ANO_CENSO', 'UNIDGEO', 'NO_CATEGORIA', 'NO_DEPENDENCIA',
       '1_CAT_FUN', '1_CAT_FUN_AI', '1_CAT_FUN_AF', '1_CAT_MED', '2_CAT_FUN',
       '2_CAT_FUN_AI', '2_CAT_FUN_AF', '2_CAT_MED', '3_CAT_FUN',
       '3_CAT_FUN_AI', '3_CAT_FUN_AF', '3_CAT_MED'],
      dtype='object')



In [68]:
RENAME_COLUMNS = {
    'NU_ANO_CENSO':'ano', 
    'UNIDGEO':'nome_uf',
    '1_CAT_FUN_AI':'taxaaprovacao_anosiniciais', 
    '1_CAT_FUN_AF':'taxaaprovacao_anosfinais', 
    '1_CAT_MED':'taxaaprovacao_ensinomedio', 
    '2_CAT_FUN_AI':'taxareprovacao_anosiniciais', 
    '2_CAT_FUN_AF':'taxareprovacao_anosfinais', 
    '2_CAT_MED':'taxareprovacao_ensinomedio', 
    '3_CAT_FUN_AI':'taxaabandono_anosiniciais', 
    '3_CAT_FUN_AF':'taxaabandono_anosfinais', 
    '3_CAT_MED' : 'taxaabandono_ensinomedio' 
}

In [69]:
def drop_unused_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols_drop = [
        col
        for col in df.columns
        if col.startswith("NO_") 
        or col.startswith("1_") 
        or col.startswith("2_") 
        or col.startswith("3_")
    ]

    return df.drop(columns=cols_drop)

dfs = {
    name: drop_unused_columns(
        df.rename(columns=RENAME_COLUMNS, errors="raise")
    )
    for name, df in dfs.items()
}

In [70]:
dfs

{'BRASIL_REGIOES_UFS ':                                             ano           nome_uf  \
 0                                          2007            Brasil   
 1                                          2007             Norte   
 2                                          2007          Nordeste   
 3                                          2007           Sudeste   
 4                                          2007               Sul   
 ..                                          ...               ...   
 492                                        2021       Mato Grosso   
 493                                        2021             Goiás   
 494                                        2021  Distrito Federal   
 495                                         NaN               NaN   
 496  Fonte: Censo da Educação Básica 2021/INEP.               NaN   
 
      taxaaprovacao_anosiniciais  taxaaprovacao_anosfinais  \
 0                          71.6                      78.7   
 1         

In [71]:
for sheet_name, df in dfs.items():
    print(f"Sheet: {sheet_name}")
    print(df.columns)  # This will print the column names of each DataFrame
    print()  # Adds a blank line for readability

Sheet: BRASIL_REGIOES_UFS 
Index(['ano', 'nome_uf', 'taxaaprovacao_anosiniciais',
       'taxaaprovacao_anosfinais', 'taxaaprovacao_ensinomedio',
       'taxareprovacao_anosiniciais', 'taxareprovacao_anosfinais',
       'taxareprovacao_ensinomedio', 'taxaabandono_anosiniciais',
       'taxaabandono_anosfinais', 'taxaabandono_ensinomedio'],
      dtype='object')



In [72]:
melted_dataframe = pd.concat(
    [
        df.pipe(
            lambda d: d.loc[(d["nome_uf"].notna()) & (d["nome_uf"] != " ")]
        )
        .pipe(
            lambda d: pd.melt(
                d,
                id_vars=["ano", "nome_uf"],
                value_vars=d.columns.difference(["ano", "nome_uf"]).tolist(),  # Convert to list
                var_name="metrica",
                value_name="valor",
            )
        )
    ]
)

In [73]:
melted_dataframe

,ano,nome_uf,metrica,valor
0,2007,Brasil,taxaabandono_anosfinais,5.1
1,2007,Norte,taxaabandono_anosfinais,7.9
2,2007,Nordeste,taxaabandono_anosfinais,9.1
3,2007,Sudeste,taxaabandono_anosfinais,3.8
4,2007,Sul,taxaabandono_anosfinais,3.6
...,...,...,...,...
4450,2021,Rio Grande do Sul,taxareprovacao_ensinomedio,1.5
4451,2021,Mato Grosso do Sul,taxareprovacao_ensinomedio,8.4
4452,2021,Mato Grosso,taxareprovacao_ensinomedio,13.4
4453,2021,Goiás,taxareprovacao_ensinomedio,1.3


In [74]:
melted_dataframe['etapa_ensino'] = melted_dataframe['metrica'].apply(
    lambda v: v.split('_')[-1])  # Extracts 'anosiniciais', 'anosfinais', or 'ensinomedio'
melted_dataframe['tipo_metrica'] = melted_dataframe['metrica'].apply(
    lambda v: v.split('_')[0])    # Extracts 'taxaaprovacao', 'taxareprovacao', 'taxaabandono'

# Pivoting the melted DataFrame to get desired columns
df_final = melted_dataframe.pivot_table(index=['ano', 'nome_uf', 'etapa_ensino'], 
                                 columns='tipo_metrica', 
                                 values='valor').reset_index()

In [75]:
RENAME_COLUMNS_MELTED = {
    'taxaabandono':'taxa_abandono', 
    'taxaaprovacao':'taxa_aprovacao',
    'taxareprovacao':'taxa_reprovacao'   
}

etapa_ensino = {
    'anosiniciais': 'Ensino Fundamental – Anos Iniciais',
    'anosfinais':'Ensino Fundamental – Anos Finais',
    'ensinomedio':'Ensino Médio Regular'
}

In [76]:
df_final

tipo_metrica,ano,nome_uf,etapa_ensino,taxaabandono,taxaaprovacao,taxareprovacao
0,2007,Acre,anosfinais,0.0,91.9,8.1
1,2007,Acre,anosiniciais,4.8,70.5,24.7
2,2007,Acre,ensinomedio,6.1,84.8,9.1
3,2007,Alagoas,anosfinais,16.3,64.8,18.9
4,2007,Alagoas,anosiniciais,9.0,62.2,28.8
...,...,...,...,...,...,...
1480,2021,São Paulo,anosiniciais,0.5,69.8,29.7
1481,2021,São Paulo,ensinomedio,3.6,94.3,2.1
1482,2021,Tocantins,anosfinais,1.2,96.2,2.6
1483,2021,Tocantins,anosiniciais,0.4,88.4,11.2


In [77]:
df_final = df_final.rename(columns=RENAME_COLUMNS_MELTED)
df_final['etapa_ensino'] = df_final['etapa_ensino'].replace(etapa_ensino)

In [78]:
bd_dir = pd.read_csv(
    '/home/vilelaluiza/dados/br_inep_sinopse_estatistica_educacao_basica/input/br_bd_diretorios_brasil.uf.csv'
)

# Perform an inner merge based on 'nome_uf' and 'nome'
df_final = pd.merge(df_final, bd_dir[['nome', 'sigla']], how='inner', left_on='nome_uf', right_on='nome')

# Rename the 'sigla' column to 'sigla_uf' and drop the 'nome' column
df_final = df_final.rename(columns={'sigla': 'sigla_uf'}).drop(columns=['nome_uf','nome'])


In [81]:
df_final['sigla_uf'].unique()

array(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT',
       'MS', 'MG', 'PR', 'PB', 'PA', 'PE', 'PI', 'RN', 'RS', 'RJ', 'RO',
       'RR', 'SC', 'SE', 'SP', 'TO'], dtype=object)

In [82]:
df_final = df_final[['ano', 'sigla_uf', 'etapa_ensino', 'taxa_aprovacao','taxa_reprovacao','taxa_abandono']]

In [83]:
path = os.path.join(
        OUTPUT, "educacao_especial_taxa_rendimento"
    )

os.makedirs(path, exist_ok=True)
df_final.astype(str).to_csv(os.path.join(path, "data.csv"), index=False)